<a href="https://colab.research.google.com/github/hfelizzola/Investigaciones-de-Operaciones-I/blob/main/flujo-redes/06_carga_datos_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de datos desde excel

El objetivo de este notebook es guiarte en el proceso de carga y gestión de los datos de tu problema desde un archivo de Excel. Para asegurar una importación fluida y una fácil manipulación, te recomendamos seguir una estructura específica para tu archivo:

* **Un solo archivo de Excel:** Centraliza todos los datos del problema en un único libro de trabajo.
* **Nombre de las columnas:** Nombra todas las columnas de la tabla usando letras minúsculas
* **Hojas separadas por parámetro:** Divide la información en hojas de cálculo dedicadas, una para cada tipo de parámetro (por ejemplo, una hoja para los costos, otra para las demandas, etc.).
* **Organización por columnas:** Dentro de cada hoja, organiza los datos de forma vertical. Cada columna debe representar un conjunto de datos único que identifique un parámetro específico.


## Ejemplo

Una empresa de fabricación necesita transportar un producto desde tres plantas a cuatro almacenes. El objetivo es minimizar el costo total de transporte.

A continuación, se presentan las tablas de datos que se usarían para resolver este problema:

1. Costos de transporte (hoja: 'costos')
Estos son los costos unitarios de transporte desde cada origen a cada destino.

<table>
<thead>
    <tr>
        <th class="text-center-left">Planta</th>
        <th>Almacen 1</th>
        <th>Almacen 2</th>
        <th>Almacen 3</th>
        <th>Almacen 4</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td class="text-center-left font-semibold">P1</td>
        <td>5</td>
        <td>2</td>
        <td>7</td>
        <td>3</td>
    </tr>
    <tr>
        <td class="text-center-left font-semibold">P2</td>
        <td>6</td>
        <td>4</td>
        <td>5</td>
        <td>8</td>
    </tr>
    <tr>
        <td class="text-center-left font-semibold">P3</td>
        <td>4</td>
        <td>7</td>
        <td>6</td>
        <td>2</td>
    </tr>
</tbody>
</table>

2. Oferta de las fábricas (hoja: 'capacidad')
Esta tabla muestra la cantidad de producto disponible en cada fábrica.

<table>
<thead>
    <tr>
        <th class="text-center-left">Planta</th>
        <th>Cantidad</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td class="text-center-left font-semibold">P1</td>
        <td>150</td>
    </tr>
    <tr>
        <td class="text-center-left font-semibold">P2</td>
        <td>100</td>
    </tr>
    <tr>
        <td class="text-center-left font-semibold">P3</td>
        <td>200</td>
    </tr>
</tbody>
</table>

3. Demanda de los almacenes (hoja: 'demanda')
Esta tabla indica la cantidad de producto requerida por cada almacén.

<table>
<thead>
    <tr>
        <th class="text-center-left">Almacen</th>
        <th>Cantidad</th>
    </tr>
</thead>
<tbody>
    <tr>
        <td class="text-center-left font-semibold">A1</td>
        <td>80</td>
    </tr>
    <tr>
        <td class="text-center-left font-semibold">A2</td>
        <td>120</td>
    </tr>
    <tr>
        <td class="text-center-left font-semibold">A3</td>
        <td>150</td>
    </tr>
    <tr>
        <td class="text-center-left font-semibold">A4</td>
        <td>100</td>
    </tr>
</tbody>
</table>

In [17]:
!pip install gamspy --quiet

In [18]:
import numpy as np
import pandas as pd
from gamspy import Container, Set, Variable, Parameter, Equation, Model, Sense, Sum, Options

In [19]:
# 1. Conectar la carpeta de googlecolab
from google.colab import drive
drive.mount('/content/drive')

# 2. Configurar directorio de trabajo
%cd /content/drive/MyDrive/Colab Notebooks/Investigacion de Operaciones I/flujo en redes
!ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Investigacion de Operaciones I/flujo en redes
03_modelo_transporte.ipynb  05_modelo_asignacion.ipynb	datos.xlsx
04_modelo_transbordo.ipynb  06_carga_datos_excel.ipynb


In [20]:
# 3. Cargar datos
costos_data = pd.read_excel('datos.xlsx', sheet_name='costos')
capacidad_data = pd.read_excel('datos.xlsx', sheet_name='capacidad')
demanda_data = pd.read_excel('datos.xlsx', sheet_name='demanda')

In [21]:
costos_data

,planta,almacen,costo
0,p1,a1,5
1,p1,a2,2
2,p1,a3,7
3,p1,a4,3
4,p2,a1,6
5,p2,a2,4
6,p2,a3,5
7,p2,a4,8
8,p3,a1,4
9,p3,a2,7


In [22]:
capacidad_data

,planta,capacidad
0,p1,150
1,p2,100
2,p3,200


In [23]:
demanda_data

,almacen,demanda
0,a1,80
1,a2,120
2,a3,150
3,a4,100


In [26]:
m = Container()

# Conjuntos
planta = Set(m, name='planta', records=capacidad_data['planta'].unique().tolist())
almacen = Set(m, name='almacen', records=demanda_data['almacen'].unique().tolist())

# Variable de decisión
x = Variable(m, name='x', type='Positive', domain=[planta,almacen])

# Parametros
capacidad = Parameter(m, name='capacidad', domain=[planta], records=capacidad_data)
demanda = Parameter(m, name='demanda', domain=[almacen], records=demanda_data)
costo = Parameter(m, name='costo', domain=[planta,almacen], records=costos_data)

# Restricciones
capacidad_max = Equation(m, name='capacidad_max', domain=[planta])
capacidad_max[planta] = Sum(almacen, x[planta,almacen]) == capacidad[planta]

demanda_max = Equation(m, name='demanda_max', domain=[almacen])
demanda_max[almacen] = Sum(planta, x[planta,almacen]) == demanda[almacen]

costo_total = Sum([planta,almacen], costo[planta,almacen] * x[planta,almacen])

# Model
modelo_transporte = Model(m, name='modelo_transporte', objective=costo_total, sense=Sense.MIN, problem='LP', equations=[capacidad_max, demanda_max])
modelo_transporte.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,1590.0,8,13,LP,CPLEX,0.001
